In [2]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import torch
import jiwer
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from datasets import Audio
from transformers import WhisperForConditionalGeneration
from dataclasses import dataclass
from typing import Any, Dict, List, Union

In [4]:
# Load the Model
model_name = "openai/whisper-small"
# Load the speech dataset
dataset = load_dataset("DTU54DL/common-accent", split="test", use_auth_token=False)

# Load the ASR model and tokenizer
model = Wav2Vec2ForCTC.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language = "english",task = "transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name, language="english", task="transcribe")

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/451 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [6]:
dataset = dataset.map(prepare_dataset)

Map:   0%|          | 0/451 [00:00<?, ? examples/s]

In [7]:
dataset = dataset.remove_columns(["accent"])

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
model = WhisperForConditionalGeneration.from_pretrained(model_name)

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [10]:
model.generation_config.language = "english"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [11]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [12]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [13]:
# load model and processor
model.config.forced_decoder_ids = None

total_wer = 0
total_samples = 0

for data in dataset:
    sample = data["audio"]
    input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features

    # generate token ids
    predicted_ids = model.generate(input_features)

    # decode token ids to text
    transcription_with_special_tokens = processor.batch_decode(predicted_ids, skip_special_tokens=False)
    transcription_without_special_tokens = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    # Calculate Word Error Rate (WER)
    wer = jiwer.wer(data["sentence"], transcription_without_special_tokens[0])
    total_wer += wer
    total_samples += 1

    print("Labels:", data["sentence"])
    print("Predicted Transcription:", transcription_without_special_tokens)
    print("Word Error Rate (WER):", wer)
    print()

average_wer = total_wer / total_samples
print("Average Word Error Rate (WER):", average_wer)


Labels: The boy swore that, every time he heard the alarm, he would heed its message.
Predicted Transcription: [' The boy swore that, every time he heard the alarm, he would heed its message.']
Word Error Rate (WER): 0.0

Labels: Their faces were hidden behind blue veils, with only their eyes showing.
Predicted Transcription: [' Their faces were hidden behind blue veils with only their eyes showing.']
Word Error Rate (WER): 0.08333333333333333

Labels: A scientist in a lab coat is using a microscope.
Predicted Transcription: [' A scientist in a lab coat is using a microscope.']
Word Error Rate (WER): 0.0

Labels: A friend to everybody is a friend to nobody.
Predicted Transcription: [' A friend to everybody is a friend to nobody.']
Word Error Rate (WER): 0.0

Labels: A group of people, one of them with a guitar, were sitting by the campfire and singing songs.
Predicted Transcription: [' A group of people, one of them with the guitar, were sitting by the campfire and singing songs.']
Wor